# Kerr Black Hole Visualization

This notebook implements a **rotating (Kerr) black hole** ray tracer using the full Kerr metric in Boyer-Lindquist coordinates.

## Key Differences from Schwarzschild

- **Frame Dragging**: Spacetime itself rotates around the black hole
- **Asymmetric Shadow**: The "D-shaped" shadow seen from edge-on views
- **Ergosphere**: Region where nothing can remain stationary
- **Different ISCO**: Prograde orbits can get much closer than retrograde

## Physics: The Kerr Metric

The Kerr metric in Boyer-Lindquist coordinates includes a frame-dragging term:

$$ds^2 = -\left(1 - \frac{2Mr}{\Sigma}\right)dt^2 - \frac{4aMr \sin^2\theta}{\Sigma}dt d\phi + \frac{\Sigma}{\Delta}dr^2 + \Sigma d\theta^2$$

Where:
- $a = J/Mc$ = spin parameter (0 to M)
- $\Sigma = r^2 + a^2\cos^2\theta$
- $\Delta = r^2 - 2Mr + a^2$
- $r_+ = M + \sqrt{M^2 - a^2}$ = event horizon

## 1. Imports and Configuration

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numba import njit, prange
import cv2
import os
import time

# Configuration
WIDTH = 800
HEIGHT = 600
FOV = 50.0
SPIN_A = 0.99  # Near-extremal spin
RS = 1.0
R_HORIZON = RS + np.sqrt(RS**2 - SPIN_A**2)

STEP_SIZE = 0.05
MAX_STEPS = 2500
CAM_DIST = 20.0
INCLINATION_DEG = 85.0  # Edge-on view

print(f"Kerr Black Hole Configuration:")
print(f"  Spin: a = {SPIN_A}")
print(f"  Event Horizon: r+ = {R_HORIZON:.3f} Rs")
print(f"  Inclination: {INCLINATION_DEG}°")

## 2. Kerr Metric Functions

The Kerr metric is more complex than Schwarzschild, requiring the inverse metric components.

In [ ]:
@njit
def get_inverse_metric(pos):
    """Calculate inverse metric g^uv for Kerr in Boyer-Lindquist coords."""
    r = pos[1]
    theta = pos[2]
    
    sin_th = np.sin(theta)
    cos_th = np.cos(theta)
    sigma = r**2 + (SPIN_A * cos_th)**2
    delta = r**2 - 2.0 * RS * r + SPIN_A**2
    
    # Inverse metric components
    inv_g_tt = -((r**2 + SPIN_A**2)**2 - delta * (SPIN_A * sin_th)**2) / (delta * sigma)
    inv_g_rr = delta / sigma
    inv_g_thth = 1.0 / sigma
    
    sin_sq = max(sin_th**2, 1e-9)  # Avoid division by zero
    inv_g_phiphi = (delta - SPIN_A**2 * sin_sq) / (delta * sigma * sin_sq)
    inv_g_tphi = -(2.0 * RS * r * SPIN_A) / (delta * sigma)  # Frame dragging!
    
    return inv_g_tt, inv_g_rr, inv_g_thth, inv_g_phiphi, inv_g_tphi

print("Metric function compiled!")

## 3. Hamiltonian Geodesic Integration

We use the Hamiltonian formulation: H = 0.5 * g^μν * p_μ * p_ν

In [ ]:
@njit
def hamiltonian(pos, p):
    """Compute Hamiltonian for geodesic."""
    g_tt, g_rr, g_thth, g_ph, g_tph = get_inverse_metric(pos)
    H = 0.5 * (g_tt * p[0]**2 + g_rr * p[1]**2 + 
               g_thth * p[2]**2 + g_ph * p[3]**2 + 
               2.0 * g_tph * p[0] * p[3])
    return H

@njit
def get_derivatives(state):
    """Hamilton's equations: dx/dλ = dH/dp, dp/dλ = -dH/dx"""
    pos = state[:4]
    p = state[4:]
    g_tt, g_rr, g_thth, g_ph, g_tph = get_inverse_metric(pos)
    
    # Velocities
    dt = g_tt * p[0] + g_tph * p[3]
    dr = g_rr * p[1]
    dth = g_thth * p[2]
    dph = g_ph * p[3] + g_tph * p[0]
    
    # Forces (numerical differentiation)
    eps = 1e-5
    pos_plus_r = np.array([pos[0], pos[1] + eps, pos[2], pos[3]])
    pos_minus_r = np.array([pos[0], pos[1] - eps, pos[2], pos[3]])
    dH_dr = (hamiltonian(pos_plus_r, p) - hamiltonian(pos_minus_r, p)) / (2*eps)
    
    pos_plus_th = np.array([pos[0], pos[1], pos[2] + eps, pos[3]])
    pos_minus_th = np.array([pos[0], pos[1], pos[2] - eps, pos[3]])
    dH_dth = (hamiltonian(pos_plus_th, p) - hamiltonian(pos_minus_th, p)) / (2*eps)
    
    dpt = 0.0
    dpphi = 0.0
    dpr = -dH_dr
    dpth = -dH_dth
    
    return np.array([dt, dr, dth, dph, dpt, dpr, dpth, dpphi])

@njit
def rk4_step_kerr(state, h):
    """4th-order Runge-Kutta step."""
    k1 = get_derivatives(state)
    k2 = get_derivatives(state + 0.5 * h * k1)
    k3 = get_derivatives(state + 0.5 * h * k2)
    k4 = get_derivatives(state + h * k3)
    return state + (h / 6.0) * (k1 + 2*k2 + 2*k3 + k4)

print("Hamiltonian integration functions compiled!")

## 4. Accretion Disk Texture

In [ ]:
@njit
def get_disk_color(r, x_hit, y_hit):
    """Procedural disk texture with spiral patterns."""
    angle = np.arctan2(y_hit, x_hit)
    spiral = np.sin(r * 3.0 + angle * 2.0)
    rings = np.sin(r * 5.0)
    
    brightness = 0.5 + 0.25 * spiral + 0.25 * rings
    beaming = 1.0 - 0.6 * (x_hit / (r + 0.1))  # Doppler effect
    brightness *= beaming
    
    norm_r = (r - R_HORIZON) / 10.0
    col_r = 1.0 * brightness
    col_g = (0.3 + 0.5 * norm_r) * brightness
    col_b = (0.1 + 0.8 * norm_r) * brightness
    
    return np.array([col_r, col_g, col_b])

print("Texture function compiled!")

## 5. Ray Marching

Note: Adaptive step size is crucial near the horizon and poles to avoid numerical artifacts.

In [ ]:
@njit
def ray_march_kerr(cam_pos_bl, ray_p_bl):
    """Trace a ray in Kerr spacetime."""
    state = np.concatenate((cam_pos_bl, ray_p_bl))
    
    for i in range(MAX_STEPS):
        prev_r = state[1]
        prev_th = state[2]
        
        # Adaptive step size
        if state[1] < 2.0 * RS:
            h_factor = 0.2  # Slow near horizon
        elif np.abs(np.sin(state[2])) < 0.15:
            h_factor = 0.1  # Slow near poles
        else:
            h_factor = 1.0
        
        current_h = STEP_SIZE * h_factor
        state = rk4_step_kerr(state, current_h)
        
        r, th = state[1], state[2]
        
        # Horizon check
        if r < R_HORIZON * 1.05:
            return np.array([0.0, 0.0, 0.0])
        
        # Escape check
        if r > CAM_DIST * 1.5:
            stars = np.sin(state[3] * 20.0) * np.sin(th * 20.0)
            if stars > 0.98:
                return np.array([1.0, 1.0, 1.0])
            return np.array([0.0, 0.0, 0.05])
        
        # Disk intersection (equatorial plane)
        if (prev_th - np.pi/2) * (th - np.pi/2) < 0:
            frac = abs(prev_th - np.pi/2) / (abs(prev_th - np.pi/2) + abs(th - np.pi/2))
            hit_r = prev_r + (r - prev_r) * frac
            
            if 2.5 * RS < hit_r < 15.0 * RS:
                hit_phi = state[3]
                x_h = hit_r * np.cos(hit_phi)
                y_h = hit_r * np.sin(hit_phi)
                return get_disk_color(hit_r, x_h, y_h)
    
    return np.array([0.0, 0.0, 0.0])

print("Ray marching function compiled!")

## 6. Rendering Function

In [ ]:
@njit(parallel=True)
def render_frame_data(width, height, cam_r, cam_theta, cam_phi, fov):
    """Render a frame from given camera position."""
    image = np.zeros((height, width, 3))
    aspect = width / height
    
    # Camera basis vectors
    cx = cam_r * np.sin(cam_theta) * np.cos(cam_phi)
    cy = cam_r * np.sin(cam_theta) * np.sin(cam_phi)
    cz = cam_r * np.cos(cam_theta)
    cam_pos_cart = np.array([cx, cy, cz])
    
    fwd = -cam_pos_cart / np.linalg.norm(cam_pos_cart)
    glob_up = np.array([0.0, 0.0, 1.0])
    if np.abs(np.dot(fwd, glob_up)) > 0.95:
        glob_up = np.array([0.0, 1.0, 0.0])
    right = np.cross(fwd, glob_up)
    right = right / np.linalg.norm(right)
    up = np.cross(right, fwd)
    
    fov_rad = np.radians(fov)
    half_h = np.tan(fov_rad/2.0)
    half_w = half_h * aspect
    
    for y in prange(height):
        for x in prange(width):
            sx = (x / width) * 2.0 - 1.0
            sy = 1.0 - (y / height) * 2.0
            
            ray_dir_cart = fwd + right * (sx * half_w) + up * (sy * half_h)
            ray_dir_cart = ray_dir_cart / np.linalg.norm(ray_dir_cart)
            
            init_pos = np.array([0.0, cam_r, cam_theta, cam_phi])
            
            # Convert ray direction to momentum
            vx, vy, vz = ray_dir_cart
            v_dot_thhat = vx*np.cos(cam_theta)*np.cos(cam_phi) + vy*np.cos(cam_theta)*np.sin(cam_phi) - vz*np.sin(cam_theta)
            v_dot_phhat = -vx*np.sin(cam_phi) + vy*np.cos(cam_phi)
            
            pr = (cx*vx + cy*vy + cz*vz) / cam_r
            pth = cam_r * v_dot_thhat
            pph = cam_r * np.sin(cam_theta) * v_dot_phhat
            
            init_p = np.array([-1.0, pr, pth, pph])
            image[y, x] = ray_march_kerr(init_pos, init_p)
    
    return image

print("Rendering function compiled!")

## 7. Render a Single Image

In [ ]:
# Compile kernels
print("Compiling JIT kernels...")
start = time.time()
_ = render_frame_data(10, 10, 20.0, np.radians(85), 0.0, 60.0)
print(f"Compilation done in {time.time() - start:.2f}s\n")

# Render
print(f"Rendering {WIDTH}×{HEIGHT} image...")
start = time.time()
img = render_frame_data(WIDTH, HEIGHT, CAM_DIST, np.radians(INCLINATION_DEG), 0.0, FOV)
print(f"Render complete in {time.time() - start:.2f}s")

# Display
plt.figure(figsize=(12, 9))
plt.imshow(np.clip(img, 0, 1))
plt.axis('off')
plt.title(f"Kerr Black Hole (a = {SPIN_A})\nFrame Dragging Creates D-Shaped Shadow", fontsize=14)
plt.tight_layout()
plt.savefig('kerr_black_hole.jpg', dpi=150, bbox_inches='tight')
print("Image saved as 'kerr_black_hole.jpg'")
plt.show()

## Summary

This notebook demonstrated:

1. ✅ **Kerr metric** in Boyer-Lindquist coordinates
2. ✅ **Frame dragging** (g^tφ term)
3. ✅ **Hamiltonian formulation** for geodesics
4. ✅ **Adaptive step sizes** near horizon and poles
5. ✅ **D-shaped shadow** characteristic of rotating black holes

### Key Differences from Schwarzschild:
- Shadow is **asymmetric** (D-shaped when viewed edge-on)
- Disk can extend closer to horizon (smaller ISCO for prograde orbits)
- Spacetime **rotation** causes frame dragging effects
- More computationally expensive (8D phase space vs 4D)

### References:
- Bardeen (1973) - Rotating black holes
- EHT Collaboration (2019) - First images of M87*
- Cunningham & Bardeen (1973) - Optical appearance of Kerr black holes